<a href="https://colab.research.google.com/github/manishwar1276/manishwar1276/blob/main/Lab%203%20part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from collections import defaultdict
import random

class TicTacToe:
    """Play TicTacToe on a 'height' by 'width' board, needing 'k' in a row to win.
    'X' plays first against 'O'."""

    def _init_(self, height=3, width=3, k=3):
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = self.k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square, k):
        """True if player has k pieces in a line through square."""
        def in_row(x, y, dx, dy):
            return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)

        return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
                   for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

class Board(defaultdict):
    """A board has the player to move, a cached utility value,
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""

    empty = '.'
    off = '#'

    def _init_(self, width=8, height=8, to_move=None, **kwds):
        super()._init_(lambda: self.empty)
        self.update(width=width, height=height, to_move=to_move, **kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def _missing_(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def _hash_(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def _repr_(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'

def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]